In [1]:
from datasets import load_dataset

/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full")
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [4]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset["train"])


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
show_random_elements(tokenized_datasets["train"], num_examples=1)


,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"I really love this place! A casual and comfortable atmosphere with awesome food and good priced wines! I love 5th and Wine for Happy Hour during the week with girlfriends-try and sit in their \""living room\"" setting and grab a few appetizers and a $5 glass of wine- I really love their Riesling. The French Fries and Bruschette's are a great way to mix sweet (Brie and Apple Bruschetta) and salty (garlic and parmesan fries). \n\nWant to get crazy? Try the Julia Child's Beef Bourguignon, a Saturday night Specialty that is Hearty and absolutely delicious!","[101, 146, 1541, 1567, 1142, 1282, 106, 138, 10243, 1105, 6062, 6814, 1114, 14918, 2094, 1105, 1363, 23812, 16728, 106, 146, 1567, 4025, 1105, 15054, 1111, 8325, 12197, 1219, 1103, 1989, 1114, 6124, 1116, 118, 2222, 1105, 3465, 1107, 1147, 165, 107, 1690, 1395, 165, 107, 3545, 1105, 6387, 170, 1374, 12647, 26883, 26542, 1105, 170, 109, 126, 2525, 1104, 4077, 118, 146, 1541, 1567, 1147, 155, 1905, 1979, 119, 1109, 1497, 13359, 1905, 1105, 139, 6208, 17374, 1566, 112, 188, 1132, 170, 1632, 1236, 1106, 5495, 4105, 113, 139, 5997, 1105, 7302, 139, 6208, 17374, 1777, 114, 1105, 6870, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/app/anaconda3/envs/llm-ft/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments
model_dir = "model/bert-base-cased-finetuned-yelp"
training_args = TrainingArguments(
    output_dir=model_dir,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    logging_steps=100,
)

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=8,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [22]:
# 小数据集
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

# 全量数据集 (太大，取10000个作比较)
full_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

# 测试集
test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(500))


In [19]:
from transformers import Trainer

small_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [20]:
small_trainer.train()
small_trainer.evaluate(test_dataset)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.718800,1.233896,0.534000
2,0.471400,1.404048,0.558000
3,0.268500,1.409490,0.580000


{'eval_loss': 1.4679054021835327,
 'eval_accuracy': 0.54,
 'eval_runtime': 6.4993,
 'eval_samples_per_second': 76.932,
 'eval_steps_per_second': 9.693,
 'epoch': 3.0}

In [23]:
full_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
full_trainer.train()
full_trainer.evaluate(test_dataset)

full_trainer.save_model(model_dir)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.881500,0.950022,0.583400
2,0.624600,1.025177,0.600000
3,0.377100,1.375666,0.603400
